In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from pathlib import Path
import yaml

In [2]:
# Настроим подключение к базе данных
CONFIG_PATH = "config.yaml"
with open(CONFIG_PATH, "r", encoding="utf-8") as config_file:
    CONFIG = yaml.load(config_file, Loader=yaml.FullLoader)

CON = create_engine(
    CONFIG['database_url']
)

def select(query: str, con: Engine = CON) -> pd.DataFrame:
    """
    Функция обобщённого sql-запроса к базе данных соцсети, содержащей три таблицы:
    1. Данные пользователей user
    2. Данные постов post
    3. Данные действий пользователей в соцсети feed
    """
    return pd.read_sql(query, con)

##### Загрузим данные из базы

In [3]:
# Посмотрим таблицу юзеров
q = """
SELECT *
FROM public.user
"""

user_data = select(q)
user_data.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [4]:
user_data.shape

(163205, 8)

In [5]:
# Теперь таблицу постов
q = """
SELECT *
FROM public.post
"""

post_data = select(q)
post_data.head()

,id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [6]:
# Наконец посмотрим на таблицу взаимодействия пользователей и постов в соцсети
q = """
SELECT count(*)
FROM public.feed_data
"""

count_feed_data = select(q)
count_feed_data.head()

,count
0,76892800


In [7]:
# Как видно, в таблице 77 миллионов записей, что превышает наши вычислительные возможности.
# Для решения этой проблемы отсортируем и пронумеруем все действия юзеров с помощью оконной функции
# и отфильтруем ранние действия юзеров и будем рассматривать только не позднее 35го по счету.

q = """
WITH q1 (timestamp, user_id, post_id, action, target, num) AS 
(
    SELECT timestamp, user_id, post_id, action, target, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) AS num
    FROM public.feed_data
    WHERE action = 'view'
)
SELECT timestamp, user_id, post_id, target
FROM q1
WHERE num <= 35
"""

feed_data = select(q)
feed_data.head()

,timestamp,user_id,post_id,target
0,2021-12-29 15:24:59,200,1773,0
1,2021-12-29 15:24:31,200,2213,1
2,2021-12-29 15:23:54,200,1122,0
3,2021-12-29 15:23:29,200,1362,0
4,2021-12-29 15:21:53,200,1541,0


In [8]:
# Получаем уже приемлемое количество записей

feed_data.shape

(5712175, 4)

##### Получим итоговый датасет для решения задачи

In [9]:
# Смёрджим датафреймы для получения итогового датасета
user_data = user_data.rename(columns={'id': 'user_id'})
post_data = post_data.rename(columns={'id': 'post_id'})

data = feed_data.merge(
    user_data, on='user_id'
).merge(
    post_data, on='post_id'
)

data.head()

,timestamp,user_id,post_id,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-12-29 15:24:59,200,1773,0,1,34,Russia,Degtyarsk,3,Android,ads,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport
1,2021-12-15 17:47:13,271,1773,0,0,36,Turkey,Gaziantep,2,Android,ads,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport
2,2021-12-25 15:41:30,279,1773,0,0,30,Russia,Vladimir,3,Android,ads,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport
3,2021-12-28 20:24:32,324,1773,0,0,35,Russia,Neman,1,Android,ads,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport
4,2021-12-25 14:11:12,363,1773,0,0,18,Russia,Belgorod,2,Android,ads,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport


In [10]:
data.shape

(5712175, 13)

In [11]:
# Сохраним датасеты для их дальнейшей обработки перед обучением модели

post_data.to_csv(CONFIG['data_folder'] + '/post_data.csv', index=False)
user_data.to_csv(CONFIG['data_folder'] + '/user_data.csv', index=False)
data.to_csv(CONFIG['datasets_folder'] + '/data.csv', index=False)